In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["HF_HOME"] = "models/"
os.environ["TORCH_HOME"] = "models/"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

In [3]:
from til_23_asr import VoiceExtractor
from til_23_asr.voice import normalize_volume
from IPython.display import Audio
import librosa
import torch
from pathlib import Path
import pydub

In [4]:
# load original and noisy speech files
# files = Path('../data/audio/val').glob('*.wav')
files = Path('/mnt/e/Downloads/simulator/orgteam1_vs_orgteam2/valid_checkpoint_8/Digits').glob('*.wav')
wavs = {}
for file in sorted(files):
    noisy, sr = librosa.load(file, sr=None)
    wavs[file.stem] = (noisy, sr)

In [5]:
extractor = VoiceExtractor(
    sr=sr,
    # spectral_gate_std_thres=1.0,
    # spectral_noise_remove=1.0,
    # spectral_n_fft=512,
    # spectral_freq_mask_hz=None,
    # spectral_time_mask_ms=None,
).to("cuda")

In [6]:
audio = Audio(normalize_volume(torch.tensor(wavs["digits3"][0])).numpy(), rate=wavs["digits3"][1])
with open('test.wav', 'wb') as f:
    f.write(audio.data)

In [ ]:
for name, (noisy, sr) in wavs.items():
    print(name)
    display(Audio(noisy, rate=sr))
    clean, new_sr, noise = extractor(
        torch.tensor(noisy, device="cuda"),
        sr,
        # skip_vol_norm=False,
        skip_demucs=True,
        # skip_spectral=False,
        # use_ori=False,
        return_noise=True,
    )
    clean = clean.numpy(force=True)
    display(Audio(clean, rate=new_sr))
    display(Audio(noise.numpy(force=True), rate=new_sr))
    

In [ ]:
demucked, d_sr = extractor._demucs(torch.tensor(noisy), sr)
display(Audio(demucked, rate=d_sr))

In [ ]:
display(Audio(extractor._spectral(demucked, d_sr)[0], rate=sr))